In [ ]:
from music21 import *
import zipfile
import glob
import os
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
with zipfile.ZipFile("/content/beeth.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/beeth_midi")

In [ ]:
os.rename("/content/beeth_midi/mond_2.mid", "/content/mond_2.mid")

In [ ]:
notes = []
for file in glob.glob("/content/beeth_midi/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [ ]:
notes

['G4',
 '0.3',
 'F#4',
 'A4',
 'F#3',
 'A3',
 'F#3',
 'G4',
 '0.3',
 'F#4',
 'A4',
 'F#3',
 'A3',
 'F#3',
 'E-5',
 'E-5',
 'D4',
 'D5',
 '7.11',
 'G4',
 'G4',
 'C4',
 'F#4',
 '9.2',
 'A4',
 'A4',
 'B4',
 '11.2',
 '7',
 '11.2',
 '7',
 'C5',
 'A4',
 'B4',
 'C5',
 'E-5',
 '5.8',
 'D5',
 'F5',
 'B3',
 'D4',
 'B3',
 'D5',
 'F5',
 'E-5',
 '5.8',
 'D5',
 'F5',
 'B3',
 'D4',
 'D5',
 'B3',
 'G#5',
 'G#5',
 'G4',
 'G5',
 '0.3',
 'C5',
 'C5',
 'F4',
 'B4',
 '2.7',
 'D5',
 'D5',
 'E-5',
 'E-4',
 '0',
 'E-4',
 '0',
 'G#4',
 '0.3',
 'G4',
 '10.1.3',
 'G#4',
 '8.0.3',
 'C5',
 '3.7.10',
 'B-4',
 'C5',
 'B-4',
 'A4',
 'B-4',
 '4.7.10',
 'C5',
 'C#5',
 '1.5.8',
 '0.4.7',
 'C5',
 'B-5',
 '0.4.7',
 'G5',
 'G5',
 '0.4.7',
 '5.8.0',
 'B-5',
 'G#5',
 'G#4',
 'G4',
 '9.0',
 'G4',
 'D2',
 'E-3',
 'F#4',
 'A4',
 'F#2',
 'A2',
 'D4',
 'A4',
 '10.2',
 'G4',
 '0.2',
 'G4',
 '9.2',
 'F#4',
 '10.2',
 'A4',
 '2.7',
 'G4',
 '9.2',
 'B-4',
 '2.6',
 'A4',
 'F#4',
 'A4',
 'C5',
 'E-5',
 'D5',
 'C5',
 'B-4',
 'A4',
 'C5',

In [ ]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(len(set(notes)))
network_output = np_utils.to_categorical(network_output)

In [ ]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(len(set(notes))))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=8, batch_size=64, callbacks=callbacks_list)

Epoch 1/8
1341/1341 [==============================] - 5769s 4s/step - loss: 4.5322
Epoch 2/8
 492/1341 [==========>...................] - ETA: 59:34 - loss: 4.5287